## Энергетический оракул
Ноутбук команды #12

Работа выполнена на основе модели LightGBM


### 1. Подготовка данных

In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random

import lightgbm as lgb
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score

from xgboost import XGBRegressor
from xgboost.callback import TrainingCallback

from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True )

import re

from tqdm import tqdm
import time

random_state = 12345
NUM_ITERATIONS = 5000

INFO: Pandarallel will run on 10 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


#### 1.1 Функции для расшифровки прогноза погоды в колонке 'weather_pred'

In [2]:
# Расшифровка прогноза в колонке 'weather_pred'

# функция формирует колонки 'cloudy', 'rainy', 'windy', 'clear', 'rain_probability', 'has_rain_probability'
# в колонках число, которое 0 при отсутсвии упоминания явления в weather_pred или степень упоминания
# функция дает в колонках номер первого списка, элемент которого есть в строке плюс 1
# списки cloudy_list, rainy_list, windy_list, clear_list можно модифицировать
# соответственно, можно экспериментировать с расположением значений в списках
# например, сейчас 'дождь', 'снег', 'д+сн' - первая степень  дождя, а 'гроз', 'ливень' - вторая
# а можно сделать снег второй, а грозу с ливнем убрать в третью
# также сделал отдельный список для "ясности", чтобы выделить 'ясно' и 'солнечно'

def in_what_list(weather, big_list):
    for list_number, small_list in enumerate(big_list):
        if any(word in weather for word in small_list):
            return list_number+1
    return 0

def weather_split2(row):
    weather = row['weather_pred']
    cloudy_list = [['проясн', 'пер.об.', 'п/об'], ['пасм', 'обл']]
    rainy_list = [['дождь', 'снег', 'д+сн'], ['гроз', 'ливень']]
    windy_list = [['вет'],['штор']]
    clear_list = [['проясн'], ['ясно'], ['солнеч']]
    numbers = re.findall(r'\d+', weather)
    cloudy = in_what_list(weather, cloudy_list)
    rainy = in_what_list(weather, rainy_list)
    windy = in_what_list(weather, windy_list)
    clear = in_what_list(weather, clear_list)
    rain_probability = 0 if len(numbers)==0 else int(numbers[0])
    has_rain_probability = int(len(numbers)==0)
    return cloudy, rainy, windy, clear, rain_probability, has_rain_probability

def fill_weather_columns(df):
    df['weather_pred'] = df['weather_pred'].fillna('')
    df['cloudy'], df['rainy'], df['windy'], df['clear'], df['rain_probability'], df['has_rain_probability'] = \
                zip(*df.apply(weather_split2, axis=1))
    return df

#### 1.2 Функции для загрузки данных о ВВП 
данные загружаются из файла 'data/VVP.csv'

Некоторые научные работы указывают на прямую связь величины потребления электричества и показателя ВВП, который отражает ситуацию в экономике. Данные по экономике публикуются различными министерствами с разной периодичностью. Для использования в работе были взяты фактические данные по ВВП с сайта investing, который агрегирует публикации Минэкономразвития. Данные за месяц побликуются с месячной задержкой, поэтому модель использует для прогнозирования данные за прошлые месяцы, которые известны.   
  
Ссылка на данные: https://ru.investing.com/economic-calendar/russian-monthly-gdp-407


In [3]:
# Функция добавляет данные о ВВП из файла 'data/VVP.csv' в датасет

def add_vvp2(data, file_source = 'data/VVP.csv'):
    """
    сырой датафрем подаем на вход
    """
    # обработаем файл с динамикой ВВП
    vvp = pd.read_csv(file_source)
    # преобразуем дату файла-источника в формат datetime64 и дропнем один столбик
    vvp['date'] = pd.to_datetime(vvp['date'], format ='%Y-%m-%d %H:%M:%S')
    vvp.drop('for_month',axis=1,inplace=True) 
    
    # обработаем основной фрейм - создадим столбец для соединения, который потом удалим
    data['date_temp'] = pd.to_datetime(data['date'], format = '%Y-%m-%d' )
    data['date_temp'] = data['date_temp'] + pd.to_timedelta(data['time'] , 'H')
    
    # соединяем основной фрейм и ВВП по дате объявления показтеля ВВП
    for idx in reversed(vvp.index):
        data.loc[data['date_temp']>=vvp.date[idx],'VVP'] = vvp.VVP_perc[idx]
        
    data.drop('date_temp',axis=1,inplace=True)   

    return data

#### 1.3 Функции для загрузки архива данных о фактической погоде
данные загружаются из файла 'data/preprocessing_loaded_table.csv'

Изначально данные для формирования таблицы "preprocessing_loaded_table" были взяты из с сайта [https://rp5.ru](https://rp5.ru/Архив_погоды_в_Храброво,_им._императрицы_Елизаветы_Петровны_(аэропорт),_METAR), где хранятся архивы погоды в аэрапорту Калининграда, за период с 31.12.2018 по 30.09.2023

Описание данных в таблице:
- Местное время в Храброво / им. императрицы Елизаветы Петровны (аэропорт) - Дата / Местное время
- T -  Темпиратура воздуха
- Po - Давление на уровне станции
- P - Давление приведённое к уровню моря
- U - Относительная влажность
- DD - Направление ветра
- Ff - Скорость ветра
- ff10 - Максимальное значение порыва ветра
- WW - Особое явление текущей погоды (осадки)
- W'W' - Явление недавней погоды, имеющее оперативное значение
- с - Общая облачность
- VV - Горизонтальная дальность видимости
- Td - Темпиратура точки росы

Данные, которые были взяты из данной таблицы и загружаются из 'data/preprocessing_loaded_table.csv':
- P - не подверглось изменению
- U - не подверглось изменению
- Td - не подверглась изменению

 WW - разделили на 4 категории:
- Нет осадков (где были пропуски)
- слабый дождь
- сильный дождь
- снег

DD - создали 4 столбца, соответствующих сторонам горизонта, которые принимали значения 0; 0.5 и 1 в зависимости от силы ветра в конкретном направлении
- N - north
- S - south
- W - west
- E - east

В дальнейшем эти данные использовались с лагом в сутки: в поля на завтрашний день записывались данные сегодняшнего.

In [4]:
# Функции для работы с данными о фактической погоде из 'data/preprocessing_loaded_table.csv'

# Кодировка информации об осадках из колонки WW
def true_weather_WW_replace(ww):
    if ww=='нет осадков':
        return 0
    elif ww=='слабый дождь':
        return 1
    elif (ww=='сильный дождь') or (ww=='снег'):
        return 2
    else:
        return 3

# Вычисление Timestamp из даты и времени
def row_plus_hours_to_index(row):
    return row['date'] + pd.to_timedelta(row['time'] , 'H')

# Функция для сдвига на сутки (в скачанном датасете разбивка по 30 мин, поэтому timeshift=48)
def shift_features_fact(df, timeshift=48):
    list_fact_columns=list(df.columns)
    list_fact_columns.remove('date_tw')
    new_df = df.copy()
    for column in list_fact_columns:
        new_df[column] = new_df[column].shift(timeshift)

    return new_df

In [5]:
# Функция для вычисления метрики mae по дням из почасовых массивов данных

def mae_day(y_true, y_pred):
    y_true_copy = pd.DataFrame(y_true).reset_index(drop=True)
    y_true_copy['day'] = y_true_copy.index // 24
    y_true_grouped = y_true_copy.groupby(by='day').sum()   
    y_pred_copy = pd.DataFrame(y_pred).reset_index(drop=True)
    y_pred_copy['day'] = y_pred_copy.index // 24
    y_pred_grouped = y_pred_copy.groupby(by='day').sum()
    
    return mean_absolute_error(y_true_grouped, y_pred_grouped)
# Функция для вычисления метрик по дням из почасовых массивов данных

def metrics_day(y_true, y_pred):
    y_true_copy = pd.DataFrame(y_true).reset_index(drop=True)
    y_true_copy['day'] = y_true_copy.index // 24
    y_true_grouped = y_true_copy.groupby(by='day').sum()   
    y_pred_copy = pd.DataFrame(y_pred).reset_index(drop=True)
    y_pred_copy['day'] = y_pred_copy.index // 24
    y_pred_grouped = y_pred_copy.groupby(by='day').sum()
    
    mae = mean_absolute_error(y_true_grouped, y_pred_grouped)
    mape = mean_absolute_percentage_error(y_true_grouped, y_pred_grouped)
    r2 = r2_score(y_true_grouped, y_pred_grouped)
    return mae, mape, r2

def metrics_hour(y_true, y_pred): 
    mae = mean_absolute_error(y_true, y_pred)
    mape = mean_absolute_percentage_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    return mae, mape, r2

#### 1.5 Чтение файлов с данными
Данные объединяются в один датасет

In [6]:
# читаем исходные датасеты и складываем в один
train_ds = pd.read_csv('data/train_dataset.csv')
test_ds = pd.read_csv('data/test_dataset.csv')
train_ds = pd.concat([train_ds, test_ds])

# запоминаем дату начала тестовых данных, потом также поступим и с закрытым датасетом
open_test_begin = pd.to_datetime(test_ds['date']).min()
open_test_end = pd.to_datetime(test_ds['date']).max() + pd.to_timedelta(1,'d')
print('начало открытого теста:', open_test_begin, '    конец открытого теста:', open_test_end)

начало открытого теста: 2023-04-01 00:00:00     конец открытого теста: 2023-08-01 00:00:00


#### 1.6 Формирование колонок с производными от даты

In [7]:
# преобразуем дату и делаем из нее колонки
train_ds['date'] = pd.to_datetime(train_ds['date'])
train_ds['year'] = train_ds['date'].dt.year
train_ds['month'] = train_ds['date'].dt.month
train_ds['day_of_week'] = train_ds['date'].dt.dayofweek
train_ds['day'] = train_ds['date'].dt.day
train_ds['day_of_year'] = train_ds['date'].dt.dayofyear

#### 1.7 Подгрузка данных о праздниках

In [8]:
# Добавление данных о праздниках из файла 'data/holidays.csv'

df_holidays = pd.read_csv('data/holidays.csv')
df_holidays['date'] = pd.to_datetime(df_holidays['date'])

# Assuming df_holidays and train_ds are your dataframes
train_ds = pd.merge(train_ds, df_holidays, on='date', how='left')

# Fill NaN values with 0
train_ds['holidays'].fillna(0, inplace=True)
train_ds['preholidays'].fillna(0, inplace=True)

# Convert to int
train_ds['holidays'] = train_ds['holidays'].astype(int)
train_ds['preholidays'] = train_ds['preholidays'].astype(int)

In [9]:
df_holidays['date'].max()

Timestamp('2023-12-31 00:00:00')

In [10]:
df_holidays[df_holidays.duplicated()]

,date,holidays,preholidays


#### 1.8 Формирование колонок со значением целевого признака в предыдущие дни

In [11]:
# Добавление колонок с временными лагами

# создаем столбец 'temp_last_day'
train_ds['temp_last_day'] = train_ds['temp'].shift(24)

# заполняем пропущенные значения в 'temp_last_day'
train_ds['temp_last_day'].fillna(method='bfill', inplace=True)

# создаем столбцы с временными лагами для 'target'
lags = [24, 48, 72, 7*24, 14*24]
for lag in lags:
    train_ds[f'target_lag_{lag}'] = train_ds['target'].shift(lag)

# заполняем пропущенные значения в столбцах с лагами
for lag in lags:
    train_ds[f'target_lag_{lag}'].fillna(0, inplace=True)

/var/folders/4y/zccbjjq17fgd73999h5g3ltr0000gn/T/ipykernel_43142/2993370237.py:7: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  train_ds['temp_last_day'].fillna(method='bfill', inplace=True)


#### 1.9 Формирование колонок с ВВП и данными о погоде посредством ранее описанных функций

In [12]:
# применяем функцию добавления ВВП
train_ds = add_vvp2(train_ds)

# Расшифровка прогноза в колонке 'weather_pred'
train_ds = fill_weather_columns(train_ds)


# Читаем файл с архивом фактической погоды
df_true_weather = pd.read_csv('data/preprocessing_loaded_table.csv')
display(df_true_weather)

# Форматируем колонки
df_true_weather['WW'] = df_true_weather['WW'].apply(true_weather_WW_replace)
df_true_weather['date'] = pd.to_datetime(df_true_weather['date'])
df_true_weather = df_true_weather.rename(columns={'date':'date_tw'})
# Применяем сдвиг на сутки, чтобы не заглядывать в будущее
df_true_weather = shift_features_fact(df_true_weather)
# Добавляем в датасет
train_ds['date_hours'] = train_ds.apply(row_plus_hours_to_index, axis=1)
train_ds = train_ds.merge(df_true_weather, left_on='date_hours', right_on='date_tw')
train_ds = train_ds.drop(['date_hours', 'date_tw'], axis=1)

,date,P,U,WW,Td,N,S,W,E
0,2018-12-31 00:00:00,763.5,100.0,слабый дождь,2.0,1.0,0.0,0.0,0.0
1,2018-12-31 00:30:00,764.3,93.0,слабый дождь,1.0,1.0,0.0,0.0,0.5
2,2018-12-31 01:00:00,764.3,93.0,слабый дождь,1.0,1.0,0.0,0.0,0.0
3,2018-12-31 01:30:00,765.0,93.0,слабый дождь,2.0,1.0,0.0,0.0,0.0
4,2018-12-31 02:00:00,765.0,93.0,нет осадков,2.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
82146,2023-09-30 21:30:00,763.5,82.0,нет осадков,12.0,0.0,0.0,1.0,0.0
82147,2023-09-30 22:00:00,763.5,82.0,нет осадков,12.0,0.5,0.0,1.0,0.0
82148,2023-09-30 22:30:00,763.5,77.0,сильный дождь,11.0,0.0,0.0,1.0,0.0
82149,2023-09-30 23:00:00,763.5,94.0,сильный дождь,13.0,0.5,0.0,1.0,0.0


#### 1.10 Формирование колонок со средними значениями цели и фактической температуры за предыдущий день и срезы по нему

In [13]:
#train_ds[['last_day_avg_target', 'last_day_avg_temp']] = train_ds[['date', 'target', 'temp']].groupby(by='date').transform('mean').shift(24)

#def mean_evening(values, evening=19):
#    return values[evening:].mean()

#evening_slices = [19, 22]
   
#for evening_slice in evening_slices:
#    train_ds[['last_evening_avg_target_'+str(evening_slice), 'last_evening_avg_temp_'+str(evening_slice)]] = \
#        train_ds[['date', 'target', 'temp']].groupby(by='date').transform(mean_evening, evening=evening_slice).shift(24)

In [14]:
train_ds[['last_day_avg_target', 'last_day_avg_temp']] = train_ds[['date', 'target', 'temp']].groupby(by='date').transform('mean').shift(24)

def mean_evening(values, evening=19):
    return values[evening:].mean()

evening_slices = [19, 22]
    
for evening_slice in evening_slices:
    train_ds[['last_evening_avg_target_'+str(evening_slice), 'last_evening_avg_temp_'+str(evening_slice)]] = \
        train_ds[['date', 'target', 'temp']].groupby(by='date').transform(mean_evening, evening=evening_slice).shift(24)

In [26]:
print(train_ds.shape)

(40027, 806)


#### 1.11 Формирование колонок с почасовыми лагами для всех сформированных ранее готовых признаков

Сначала готовим список названий

In [16]:
# Отбираем признаки. Все лишние колонки здесь отбрасываем, кроме 'date', которую уберем позже 

feature_cols = list(train_ds.columns)

# выбрасываем взгляд в прошлое и расшифрованную погоду
drop_list = ['target', 'day_of_year', 'weather_pred', 'weather_fact', 'temp']

# выбрасываем признаки, найденные процедурно в процессе оптимизации
# КОМАНДЕ: здесь можно добавлять признаки на выброс с целью оптимизации
drop_list = drop_list + ['target_lag_48', 'target_lag_168'] #, 'temp_pred'] #, 'target_lag_336'] 

for name in drop_list:
    feature_cols.remove(name)

Потом добавляем лаги

In [17]:
FEATURE_WINDOW_SIZE = 24
feature_cols_no_date = feature_cols.copy()
feature_cols_no_date.remove('date')


for lag in range(1,FEATURE_WINDOW_SIZE):
    for column in feature_cols_no_date:
        train_ds[column+'_'+str(lag)] = train_ds[column].shift(lag)

/var/folders/4y/zccbjjq17fgd73999h5g3ltr0000gn/T/ipykernel_43142/4047160460.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_ds[column+'_'+str(lag)] = train_ds[column].shift(lag)
/var/folders/4y/zccbjjq17fgd73999h5g3ltr0000gn/T/ipykernel_43142/4047160460.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_ds[column+'_'+str(lag)] = train_ds[column].shift(lag)
/var/folders/4y/zccbjjq17fgd73999h5g3ltr0000gn/T/ipykernel_43142/4047160460.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the res

#### 1.11 Формирование колонок с лагами для цели и фактической температуры

Заполняем значениями NaN все поля этих лагов, которые относятся к текущему дню, чтобы не допустить утечки. Т.е. для каждого часа используем только лаги, которые превосходят номер часа.

In [18]:
target_lags=[1, 5, 9]

for lag in target_lags:
    train_ds['target_'+str(lag)] = train_ds.target.shift(lag).where(train_ds['time']<lag, np.NaN)
    train_ds['temp_'+str(lag)] = train_ds['temp'].shift(lag).where(train_ds['time']<lag, np.NaN)

/var/folders/4y/zccbjjq17fgd73999h5g3ltr0000gn/T/ipykernel_43142/1660665095.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_ds['target_'+str(lag)] = train_ds.target.shift(lag).where(train_ds['time']<lag, np.NaN)
/var/folders/4y/zccbjjq17fgd73999h5g3ltr0000gn/T/ipykernel_43142/1660665095.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_ds['temp_'+str(lag)] = train_ds['temp'].shift(lag).where(train_ds['time']<lag, np.NaN)
/var/folders/4y/zccbjjq17fgd73999h5g3ltr0000gn/T/ipykernel_43142/1660665095.py:4: Performan

#### 1.12 Демонстрация сформированного датасета

In [19]:
# Итоговый набор колонок
train_ds.columns

Index(['date', 'time', 'target', 'temp', 'temp_pred', 'weather_pred',
       'weather_fact', 'year', 'month', 'day_of_week',
       ...
       'last_evening_avg_target_19_23', 'last_evening_avg_temp_19_23',
       'last_evening_avg_target_22_23', 'last_evening_avg_temp_22_23',
       'target_1', 'temp_1', 'target_5', 'temp_5', 'target_9', 'temp_9'],
      dtype='object', length=806)

In [20]:
train_ds.head()

,date,time,target,temp,temp_pred,weather_pred,weather_fact,year,month,day_of_week,...,last_evening_avg_target_19_23,last_evening_avg_temp_19_23,last_evening_avg_target_22_23,last_evening_avg_temp_22_23,target_1,temp_1,target_5,temp_5,target_9,temp_9
0,2019-01-01,0,481.510,2.9,2.0,"пасм, ветер",ветер,2019,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019-01-01,1,462.872,2.9,2.0,"пасм, ветер",ветер,2019,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019-01-01,2,449.718,2.9,2.0,"пасм, ветер",ветер,2019,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019-01-01,3,430.908,4.3,2.0,"пасм, ветер","ветер, пасм",2019,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019-01-01,4,415.163,4.3,2.0,"пасм, ветер","ветер, пасм",2019,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### 1.13 Исключение лишних колонок

In [19]:
# Отбираем признаки. Список формируем заново из всех текущих колонок.
feature_cols = list(train_ds.columns)

# Отбрасываем колонки из ранее заготовленного списка на выброс. На этом этапе уходят колонки с утечками.
for name in drop_list:
    feature_cols.remove(name)

feature_cols

['date',
 'time',
 'temp_pred',
 'year',
 'month',
 'day_of_week',
 'day',
 'holidays',
 'preholidays',
 'temp_last_day',
 'target_lag_24',
 'target_lag_72',
 'target_lag_336',
 'VVP',
 'cloudy',
 'rainy',
 'windy',
 'clear',
 'rain_probability',
 'has_rain_probability',
 'P',
 'U',
 'WW',
 'Td',
 'N',
 'S',
 'W',
 'E',
 'last_day_avg_target',
 'last_day_avg_temp',
 'last_evening_avg_target_19',
 'last_evening_avg_temp_19',
 'last_evening_avg_target_22',
 'last_evening_avg_temp_22',
 'time_1',
 'temp_pred_1',
 'year_1',
 'month_1',
 'day_of_week_1',
 'day_1',
 'holidays_1',
 'preholidays_1',
 'temp_last_day_1',
 'target_lag_24_1',
 'target_lag_72_1',
 'target_lag_336_1',
 'VVP_1',
 'cloudy_1',
 'rainy_1',
 'windy_1',
 'clear_1',
 'rain_probability_1',
 'has_rain_probability_1',
 'P_1',
 'U_1',
 'WW_1',
 'Td_1',
 'N_1',
 'S_1',
 'W_1',
 'E_1',
 'last_day_avg_target_1',
 'last_day_avg_temp_1',
 'last_evening_avg_target_19_1',
 'last_evening_avg_temp_19_1',
 'last_evening_avg_target_22_1'

#### 1.13.1 Аугументации MIXUP

##### Обозначаем все регрессионные признаки которые в последствии мы изменим

In [20]:
feature_aug = [
'temp_pred',
'temp_last_day',
'target_lag_24',
'target_lag_72',
'target_lag_336',
'VVP',
'P',
'U',
'Td',
'last_day_avg_target',
'last_day_avg_temp',
'last_evening_avg_target_19',
'last_evening_avg_temp_19',
'last_evening_avg_target_22',
'last_evening_avg_temp_22',
'temp_pred_1',
'temp_last_day_1',
'target_lag_24_1',
'target_lag_72_1',
'target_lag_336_1',
'VVP_1',
'P_1',
'U_1',
'Td_1',
'last_day_avg_target_1',
'last_day_avg_temp_1',
'last_evening_avg_target_19_1',
'last_evening_avg_temp_19_1',
'last_evening_avg_target_22_1',
'last_evening_avg_temp_22_1',
'temp_pred_2',
'temp_last_day_2',
'target_lag_24_2',
'target_lag_72_2',
'target_lag_336_2',
'VVP_2',
'P_2',
'U_2',
'Td_2',
'last_day_avg_target_2',
'last_day_avg_temp_2',
'last_evening_avg_target_19_2',
'last_evening_avg_temp_19_2',
'last_evening_avg_target_22_2',
'last_evening_avg_temp_22_2',
'temp_pred_3',
'temp_last_day_3',
'target_lag_24_3',
'target_lag_72_3',
'target_lag_336_3',
'VVP_3',
'P_3',
'U_3',
'Td_3',
'last_day_avg_target_3',
'last_day_avg_temp_3',
'last_evening_avg_target_19_3',
'last_evening_avg_temp_19_3',
'last_evening_avg_target_22_3',
'last_evening_avg_temp_22_3',
'temp_pred_4',
'temp_last_day_4',
'target_lag_24_4',
'target_lag_72_4',
'target_lag_336_4',
'VVP_4',
'P_4',
'U_4',
'Td_4',
'last_day_avg_target_4',
'last_day_avg_temp_4',
'last_evening_avg_target_19_4',
'last_evening_avg_temp_19_4',
'last_evening_avg_target_22_4',
'last_evening_avg_temp_22_4',
'temp_pred_5',
'temp_last_day_5',
'target_lag_24_5',
'target_lag_72_5',
'target_lag_336_5',
'VVP_5',
'P_5',
'U_5',
'Td_5',
'last_day_avg_target_5',
'last_day_avg_temp_5',
'last_evening_avg_target_19_5',
'last_evening_avg_temp_19_5',
'last_evening_avg_target_22_5',
'last_evening_avg_temp_22_5',
'temp_pred_6',
'temp_last_day_6',
'target_lag_24_6',
'target_lag_72_6',
'target_lag_336_6',
'VVP_6',
'P_6',
'U_6',
'Td_6',
'last_day_avg_target_6',
'last_day_avg_temp_6',
'last_evening_avg_target_19_6',
'last_evening_avg_temp_19_6',
'last_evening_avg_target_22_6',
'last_evening_avg_temp_22_6',
'temp_pred_7',
'temp_last_day_7',
'target_lag_24_7',
'target_lag_72_7',
'target_lag_336_7',
'VVP_7',
'P_7',
'U_7',
'Td_7',
'last_day_avg_target_7',
'last_day_avg_temp_7',
'last_evening_avg_target_19_7',
'last_evening_avg_temp_19_7',
'last_evening_avg_target_22_7',
'last_evening_avg_temp_22_7',
'temp_pred_8',
'temp_last_day_8',
'target_lag_24_8',
'target_lag_72_8',
'target_lag_336_8',
'VVP_8',
'P_8',
'U_8',
'Td_8',
'last_day_avg_target_8',
'last_day_avg_temp_8',
'last_evening_avg_target_19_8',
'last_evening_avg_temp_19_8',
'last_evening_avg_target_22_8',
'last_evening_avg_temp_22_8',
'temp_pred_9',
'temp_last_day_9',
'target_lag_24_9',
'target_lag_72_9',
'target_lag_336_9',
'VVP_9',
'P_9',
'U_9',
'Td_9',
'last_day_avg_target_9',
'last_day_avg_temp_9',
'last_evening_avg_target_19_9',
'last_evening_avg_temp_19_9',
'last_evening_avg_target_22_9',
'last_evening_avg_temp_22_9',
'temp_pred_10',
'temp_last_day_10',
'target_lag_24_10',
'target_lag_72_10',
'target_lag_336_10',
'VVP_10',
'P_10',
'U_10',
'Td_10',
'last_day_avg_target_10',
'last_day_avg_temp_10',
'last_evening_avg_target_19_10',
'last_evening_avg_temp_19_10',
'last_evening_avg_target_22_10',
'last_evening_avg_temp_22_10',
'temp_pred_11',
'temp_last_day_11',
'target_lag_24_11',
'target_lag_72_11',
'target_lag_336_11',
'VVP_11',
'P_11',
'U_11',
'Td_11',
'last_day_avg_target_11',
'last_day_avg_temp_11',
'last_evening_avg_target_19_11',
'last_evening_avg_temp_19_11',
'last_evening_avg_target_22_11',
'last_evening_avg_temp_22_11',
'temp_pred_12',
'temp_last_day_12',
'target_lag_24_12',
'target_lag_72_12',
'target_lag_336_12',
'VVP_12',
'P_12',
'U_12',
'Td_12',
'last_day_avg_target_12',
'last_day_avg_temp_12',
'last_evening_avg_target_19_12',
'last_evening_avg_temp_19_12',
'last_evening_avg_target_22_12',
'last_evening_avg_temp_22_12',
'temp_pred_13',
'temp_last_day_13',
'target_lag_24_13',
'target_lag_72_13',
'target_lag_336_13',
'VVP_13',
'P_13',
'U_13',
'Td_13',
'last_day_avg_target_13',
'last_day_avg_temp_13',
'last_evening_avg_target_19_13',
'last_evening_avg_temp_19_13',
'last_evening_avg_target_22_13',
'last_evening_avg_temp_22_13',
'temp_pred_14',
'temp_last_day_14',
'target_lag_24_14',
'target_lag_72_14',
'target_lag_336_14',
'VVP_14',
'P_14',
'U_14',
'Td_14',
'last_day_avg_target_14',
'last_day_avg_temp_14',
'last_evening_avg_target_19_14',
'last_evening_avg_temp_19_14',
'last_evening_avg_target_22_14',
'last_evening_avg_temp_22_14',
'temp_pred_15',
'temp_last_day_15',
'target_lag_24_15',
'target_lag_72_15',
'target_lag_336_15',
'VVP_15',
'P_15',
'U_15',
'Td_15',
'last_day_avg_target_15',
'last_day_avg_temp_15',
'last_evening_avg_target_19_15',
'last_evening_avg_temp_19_15',
'last_evening_avg_target_22_15',
'last_evening_avg_temp_22_15',
'temp_pred_16',
'temp_last_day_16',
'target_lag_24_16',
'target_lag_72_16',
'target_lag_336_16',
'VVP_16',
'P_16',
'U_16',
'Td_16',
'last_day_avg_target_16',
'last_day_avg_temp_16',
'last_evening_avg_target_19_16',
'last_evening_avg_temp_19_16',
'last_evening_avg_target_22_16',
'last_evening_avg_temp_22_16',
'temp_pred_17',
'temp_last_day_17',
'target_lag_24_17',
'target_lag_72_17',
'target_lag_336_17',
'VVP_17',
'P_17',
'U_17',
'Td_17',
'last_day_avg_target_17',
'last_day_avg_temp_17',
'last_evening_avg_target_19_17',
'last_evening_avg_temp_19_17',
'last_evening_avg_target_22_17',
'last_evening_avg_temp_22_17',
'temp_pred_18',
'temp_last_day_18',
'target_lag_24_18',
'target_lag_72_18',
'target_lag_336_18',
'VVP_18',
'P_18',
'U_18',
'Td_18',
'last_day_avg_target_18',
'last_day_avg_temp_18',
'last_evening_avg_target_19_18',
'last_evening_avg_temp_19_18',
'last_evening_avg_target_22_18',
'last_evening_avg_temp_22_18',
'temp_pred_19',
'temp_last_day_19',
'target_lag_24_19',
'target_lag_72_19',
'target_lag_336_19',
'VVP_19',
'P_19',
'U_19',
'Td_19',
'last_day_avg_target_19',
'last_day_avg_temp_19',
'last_evening_avg_target_19_19',
'last_evening_avg_temp_19_19',
'last_evening_avg_target_22_19',
'last_evening_avg_temp_22_19',
'temp_pred_20',
'temp_last_day_20',
'target_lag_24_20',
'target_lag_72_20',
'target_lag_336_20',
'VVP_20',
'P_20',
'U_20',
'Td_20',
'last_day_avg_target_20',
'last_day_avg_temp_20',
'last_evening_avg_target_19_20',
'last_evening_avg_temp_19_20',
'last_evening_avg_target_22_20',
'last_evening_avg_temp_22_20',
'temp_pred_21',
'temp_last_day_21',
'target_lag_24_21',
'target_lag_72_21',
'target_lag_336_21',
'VVP_21',
'P_21',
'U_21',
'Td_21',
'last_day_avg_target_21',
'last_day_avg_temp_21',
'last_evening_avg_target_19_21',
'last_evening_avg_temp_19_21',
'last_evening_avg_target_22_21',
'last_evening_avg_temp_22_21',
'temp_pred_22',
'temp_last_day_22',
'target_lag_24_22',
'target_lag_72_22',
'target_lag_336_22',
'VVP_22',
'P_22',
'U_22',
'Td_22',
'last_day_avg_target_22',
'last_day_avg_temp_22',
'last_evening_avg_target_19_22',
'last_evening_avg_temp_19_22',
'last_evening_avg_target_22_22',
'last_evening_avg_temp_22_22',
'temp_pred_23',
'temp_last_day_23',
'target_lag_24_23',
'target_lag_72_23',
'target_lag_336_23',
'VVP_23',
'P_23',
'U_23',
'Td_23',
'last_day_avg_target_23',
'last_day_avg_temp_23',
'last_evening_avg_target_19_23',
'last_evening_avg_temp_19_23',
'last_evening_avg_target_22_23',
'last_evening_avg_temp_22_23',
'target_1',
'temp_1',
'target_5',
'temp_5',
'target_9',
'temp_9']


In [21]:
feature_target = feature_cols +['target'] 
#feature_target.remove('date')
print(feature_target)

train_ds[feature_aug].dtypes

['date', 'time', 'temp_pred', 'year', 'month', 'day_of_week', 'day', 'holidays', 'preholidays', 'temp_last_day', 'target_lag_24', 'target_lag_72', 'target_lag_336', 'VVP', 'cloudy', 'rainy', 'windy', 'clear', 'rain_probability', 'has_rain_probability', 'P', 'U', 'WW', 'Td', 'N', 'S', 'W', 'E', 'last_day_avg_target', 'last_day_avg_temp', 'last_evening_avg_target_19', 'last_evening_avg_temp_19', 'last_evening_avg_target_22', 'last_evening_avg_temp_22', 'time_1', 'temp_pred_1', 'year_1', 'month_1', 'day_of_week_1', 'day_1', 'holidays_1', 'preholidays_1', 'temp_last_day_1', 'target_lag_24_1', 'target_lag_72_1', 'target_lag_336_1', 'VVP_1', 'cloudy_1', 'rainy_1', 'windy_1', 'clear_1', 'rain_probability_1', 'has_rain_probability_1', 'P_1', 'U_1', 'WW_1', 'Td_1', 'N_1', 'S_1', 'W_1', 'E_1', 'last_day_avg_target_1', 'last_day_avg_temp_1', 'last_evening_avg_target_19_1', 'last_evening_avg_temp_19_1', 'last_evening_avg_target_22_1', 'last_evening_avg_temp_22_1', 'time_2', 'temp_pred_2', 'year_2'

temp_pred         float64
temp_last_day     float64
target_lag_24     float64
target_lag_72     float64
target_lag_336    float64
                   ...   
temp_1            float64
target_5          float64
temp_5            float64
target_9          float64
temp_9            float64
Length: 366, dtype: object

In [27]:
def augment_row(df_to_augment, features_to_augment, alpha, i, random_1):


    random.seed(random_1*i) # Постоянно меняем random seed что бы аугументации не повторялись

    random_sample1 = random.randint(0000, 50000)
    random_sample2 = random.randint(50001, 100000)

    df_sample1 = df_to_augment.sample(frac=1,
                                     random_state=random_sample1
                                     )
    df_sample2 = df_to_augment.sample(frac=1,
                                     random_state = random_sample2
                                     )

    lmbda = np.random.beta(alpha, alpha)

    df_mixup_sample = df_sample1.copy()
    df_mixup_sample[features_to_augment] = df_sample1[features_to_augment] * lmbda + df_sample2[features_to_augment] * (1 - lmbda)
        
    other_features = list(set(df_to_augment.columns) - set(features_to_augment))
    df_mixup_sample[other_features] = df_sample1[other_features]

    return df_mixup_sample

def mixup(df, alpha, features_to_augment, n_augmentations):
    random.seed(random_state) #random.seed(42)
    random_1 = random.randint(500, 9500)

    
    df_to_augment = df[(df['date'] < open_test_begin)] #  датасет будет подвергнут ауггументации
    df_to_keep = df[(df['date'] >= open_test_begin)]  # Тестовая часть, не будет аугументированна

    df_mixup = pd.concat([augment_row(df_to_augment, features_to_augment, alpha, i, random_1) for i in range(n_augmentations)]).parallel_apply(lambda x: x)

    df_final = pd.concat([ df_mixup,
                           df_to_augment,  #нужно раскомитить если обучение с нуля
                           df_to_keep])

    return df_final


n_augmentations = 27 # Количество ауггументаций
alpha=0.3 #15   # Влияет на сколько изменятся наши данные
n_frac = 1  # какую часть датасета мы берем для изменений, в данный момент функция не принимает этот параметр
train_ds_mixup = mixup(train_ds[feature_target], alpha, feature_aug, n_augmentations)
FEATURES = '_Aug_27_alpha_3' # Дополнительная информация которую мы записываем во время экспериментов
num = 6 #номер модели

##### Проверка нового датасета

* Смотрим размер
* Обязательно сбрасываем индекс, что бы модели воспринимали новые данные

In [29]:
print(train_ds[feature_target].shape, train_ds_mixup.shape)
train_ds_mixup.reset_index(inplace=True) 

(40027, 800) (1041943, 800)


/var/folders/4y/zccbjjq17fgd73999h5g3ltr0000gn/T/ipykernel_43142/1146379950.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_ds_mixup.reset_index(inplace=True)


#### 1.14 Выделение наборов данных для обучения, валидации и тестирования

Выделялось два набора данных для обучения и валидации:
1. Обучение на данных с 2019 по 2021 с валидацией на 2022
2. Обучение на данных с 2019 по 2022 с валидацией на первом квартале 2023

Первый набор позволяет оценить влияние сезонности на обучение и предсказания, второй позволяет обучить модель на большем объеме данных и на более актуальных данных.

In [30]:
# Формируем набор датасетов для обучения и проверки

# здесь мы используем аугументированный dataset

features = train_ds_mixup[feature_cols] # train_ds[feature_cols]
target = train_ds_mixup['target'] # train_ds['target']

# Функция для выделения временных интервалов из таблиц признаков и целей
# на этом этапе отбрасываем колонку 'date'
def features_interval(features, target, date1, date2):
    features_interval = features[ (features['date']>=date1) & (features['date']<date2) ]
    target_interval = target[features_interval.index]
    features_interval = features_interval.drop('date', axis=1)
    return features_interval, target_interval

# для первичного подбора гиперпараметров будем обучать на 19-21 годах, валидировать по 2022
features_train, target_train = features_interval(features, target, '2019-01-01', '2022-01-01')
features_valid, target_valid = features_interval(features, target, '2022-01-01', '2023-01-01')

# отбор признаков будем производить, обучая на 19-22 и проверяя по первому кварталу 2023
# с дополнительным контролем на вариантах из первичного обучения
features_2022, target_2022 = features_interval(features, target, '2019-01-01', '2023-01-01')
features_2023, target_2023 = features_interval(features, target, '2023-01-01', open_test_begin)

# для проверки на тестовой выборке будем учиться на всем тренировочном датасете
features_all_train, target_all_train = features_interval(features, target, '2019-01-01', open_test_begin)
features_open_test, target_open_test = features_interval(features, target, open_test_begin, open_test_end)

# формируем наборы данных по кварталам 2022 года, чтобы посмотреть по ним метрику отдельно
dates = ['2022-01-01', '2022-04-01', '2022-07-01', '2022-10-01', '2023-01-01']
quarters = []
for i in range(4):
    f, t = features_interval(features, target, dates[i], dates[i+1])
    quarters.append({'features':f, 'target':t})

### 2. Обучение моделей

В настоящей работе обучаются модели LightGBM и XGBoost, финальное предсказание получается усреднением результатов.

#### 2.1 Гиперпараметры LightGBM
Были подобраны следующие значения гиперпараметров:

In [31]:
# Параметры LGBM без ауггументации
params_lgbm = {'num_leaves':15, 'learning_rate':0.02, 'feature_fraction':1, 'num_iterations':NUM_ITERATIONS, 'random_state':random_state, 'objective':'regression_l1', 'n_jobs':-1}

In [35]:
# Параметры LGBM для обучения с ауггументации
params_lgbm = {'num_leaves': 34, 'min_child_samples': 16, 
               'max_depth': 8, 'learning_rate': 0.012, 
               'min_sum_hessian_in_leaf': 1e-4,
               'objective': 'regression_l1', 'feature_fraction': 0.9574152630927155,
               'n_jobs':-1, 'num_iterations':10000
                }

### 3 Проверка метрик на тестовом датасете
##### 3.1 Модель LightGBM

In [36]:
# Проверка метрики лучшей модели на тестовом датасете
# Здесь обучаем на всем тренировочном датасете


lgbm_model_all_train = lgb.LGBMRegressor(**params_lgbm)
lgbm_model_all_train.fit(features_all_train, target_all_train)

/opt/anaconda3/envs/p311/lib/python3.11/site-packages/lightgbm/engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] min_sum_hessian_in_leaf is set=0.0001, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=0.0001
[LightGBM] [Warning] feature_fraction is set=0.9574152630927155, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9574152630927155
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=0.0001, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=0.0001
[LightGBM] [Warning] feature_fraction is set=0.9574152630927155, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9574152630927155
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.676412 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 75507
[LightGBM] [Info] Number of data points in the train set: 1039024, number of used features: 798
[LightGBM] [Info] Start training from sc

LGBMRegressor(feature_fraction=0.9574152630927155, learning_rate=0.012,
              max_depth=8, min_child_samples=16, min_sum_hessian_in_leaf=0.0001,
              n_jobs=-1, num_iterations=10000, num_leaves=34,
              objective='regression_l1')

In [43]:
l_predict_train = lgbm_model_all_train.predict(features_all_train)
l_predict_test = lgbm_model_all_train.predict(features_open_test)

mae_train, mape_train, r2_train = metrics_hour(target_all_train, l_predict_train)
mae_open_test, mape_open_test, r2_open_test = metrics_hour(target_open_test, l_predict_test)


results = pd.DataFrame([[f'тренировочная LGBM {FEATURES}', mae_train, mape_train, r2_train], [f'тестовая LGBM {FEATURES}', mae_open_test, mape_open_test, r2_open_test]], 
             columns=('Выборка', 'MAE', 'MAPE', 'R2'))
results

[LightGBM] [Warning] min_sum_hessian_in_leaf is set=0.0001, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=0.0001
[LightGBM] [Warning] feature_fraction is set=0.9574152630927155, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9574152630927155
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=0.0001, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=0.0001
[LightGBM] [Warning] feature_fraction is set=0.9574152630927155, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9574152630927155


/opt/anaconda3/envs/p311/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/anaconda3/envs/p311/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/opt/anaconda3/envs/p311/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/anaconda3/envs/p311/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_

,Выборка,MAE,MAPE,R2
0,тренировочная LGBM _Aug_27_alpha_3,3.403002,0.007062,0.996599
1,тестовая LGBM _Aug_27_alpha_3,6.092151,0.014274,0.985890


In [44]:
# получение важности признаков
importance = lgbm_model_all_train.feature_importances_
feature_name = features_open_test.columns
# создание DataFrame
importance_df_lgbm = pd.DataFrame({'feature': feature_name, 'importance': importance})
# сортировка по важности
importance_df_lgbm = importance_df_lgbm.sort_values(by='importance', ascending=False)

In [45]:
importance_df_lgbm

,feature,importance
9,target_lag_24,5168
11,target_lag_336,3262
10,target_lag_72,2998
42,target_lag_24_1,2148
1,temp_pred,2112
...,...,...
464,year_14,2
332,year_10,1
271,preholidays_8,1
469,preholidays_14,1


##### 3.2 Модель XGBoost

In [46]:
drop_list = ['preholidays',
            #'has_rain_probability', 
            # 'W', 'E'
            ]
n_values = range(1, 24)
preholidays = ['preholidays_{}'.format(n) for n in n_values]
#has_rain = ['has_rain_probability_{}'.format(n) for n in n_values]
#W_wind = ['W_{}'.format(n) for n in n_values]
#E_wind = ['E_{}'.format(n) for n in n_values]

drop_list = drop_list + preholidays #+ has_rain + W_wind + E_wind

feat_xgb_train = features_all_train.drop(columns=drop_list)
feat_xgb_test = features_open_test.drop(columns=drop_list)
#feat_xgb_train.columns, feat_xgb_test.columns

In [47]:
xgb_model = XGBRegressor(
    max_depth=7,
    n_estimators=1190, #n_estimators=195, #
    learning_rate=0.009, #learning_rate=0.1, #
    tree_method='exact',
    objective='reg:squarederror',
    eval_metric='rmse',
    gamma=2,
    colsample_bytree=1,
    random_state=random_state

)

In [48]:
# Проверка метрики лучшей модели на тестовом датасете


class IterationInfoCallback(TrainingCallback):
    def __init__(self):
        self.start_time = time.time()

    def after_iteration(self, model, epoch, evals_log):
        print('Iteration:', epoch, 'Time for last iteration:', self.start_time - time.time())
        self.start_time = time.time()
        return False



xgb_model_all_train = xgb_model.fit(feat_xgb_train, target_all_train, callbacks=[IterationInfoCallback()])
xgb_predict_test = xgb_model_all_train.predict(feat_xgb_test)
xgb_predict_train = xgb_model_all_train.predict(feat_xgb_train)


/opt/anaconda3/envs/p311/lib/python3.11/site-packages/xgboost/sklearn.py:885: UserWarning: `callbacks` in `fit` method is deprecated for better compatibility with scikit-learn, use `callbacks` in constructor or`set_params` instead.
  warnings.warn(


Iteration: 0 Time for last iteration: -26.525530099868774
Iteration: 1 Time for last iteration: -11.478789806365967
Iteration: 2 Time for last iteration: -11.780966997146606
Iteration: 3 Time for last iteration: -12.095326900482178
Iteration: 4 Time for last iteration: -11.412700176239014
Iteration: 5 Time for last iteration: -12.471737146377563
Iteration: 6 Time for last iteration: -12.06488299369812
Iteration: 7 Time for last iteration: -11.979055881500244
Iteration: 8 Time for last iteration: -11.368681907653809
Iteration: 9 Time for last iteration: -11.847079277038574
Iteration: 10 Time for last iteration: -11.418037176132202
Iteration: 11 Time for last iteration: -11.452046155929565
Iteration: 12 Time for last iteration: -11.389868021011353
Iteration: 13 Time for last iteration: -11.747148990631104
Iteration: 14 Time for last iteration: -11.650840997695923
Iteration: 15 Time for last iteration: -11.413532257080078
Iteration: 16 Time for last iteration: -11.475651741027832
Iteratio

In [49]:
mae_train, mape_train, r2_train = metrics_hour(target_all_train, xgb_predict_train )
mae_open_test, mape_open_test, r2_open_test = metrics_hour(target_open_test, xgb_predict_test )

results = pd.concat([results,
pd.DataFrame([[f'тренировочная XGB {FEATURES}', mae_train, mape_train, r2_train], [f'тестовая XGB {FEATURES}', mae_open_test, mape_open_test, r2_open_test]], 
             columns=('Выборка', 'MAE', 'MAPE', 'R2'))
 ])

results

/opt/anaconda3/envs/p311/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/anaconda3/envs/p311/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/opt/anaconda3/envs/p311/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/anaconda3/envs/p311/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_

,Выборка,MAE,MAPE,R2
0,тренировочная LGBM _Aug_27_alpha_3,3.403002,0.007062,0.996599
1,тестовая LGBM _Aug_27_alpha_3,6.092151,0.014274,0.985890
0,тренировочная XGB _Aug_27_alpha_3,3.828241,0.008054,0.997552
1,тестовая XGB _Aug_27_alpha_3,6.141930,0.014345,0.984693


In [50]:
from catboost import CatBoostRegressor

In [51]:
catboost_model = CatBoostRegressor(
    iterations=10000,
    learning_rate=0.015,
    depth=6,
    subsample=0.65,
    colsample_bylevel=0.95,
    min_data_in_leaf=51,
    l2_leaf_reg=9
)

catboost_model.fit(feat_xgb_train , target_all_train)
"""
params_catboost = {
    'iterations': [100, 200, 300, 500, 1000, 3000, 10000],
    'learning_rate': [0.001, 0.005, 0.01, 0.015, 0.02, 0.03, 0.05, 0.1],
    'depth': range(1, 10, 1),
    'subsample': [x / 100.0 for x in range(5, 100, 10)],
    'colsample_bylevel': [x / 100.0 for x in range(5, 100, 10)],
    'min_data_in_leaf': range(1, 100, 10),
    'l2_leaf_reg': [1, 3, 5, 7, 9],
    #'min_child_samples': [1, 4, 8, 16, 32]
}

grid_search_catboost = RandomizedSearchCV(
    catboost_model,
    cv = TimeSeriesSplit(n_splits=4),
    param_distributions=params_catboost,
    n_jobs=-1,
    random_state=random_state,
    scoring='neg_mean_absolute_error'
)

grid_search_catboost.fit(features_train, target_train)"""

0:	learn: 100.0859457	total: 413ms	remaining: 1h 8m 53s
1:	learn: 98.7146801	total: 737ms	remaining: 1h 1m 23s
2:	learn: 97.3672413	total: 1.1s	remaining: 1h 1m 20s
3:	learn: 96.0436854	total: 1.48s	remaining: 1h 1m 29s
4:	learn: 94.7346729	total: 1.83s	remaining: 1h 1m 6s
5:	learn: 93.4447521	total: 2.22s	remaining: 1h 1m 39s
6:	learn: 92.1675048	total: 2.6s	remaining: 1h 1m 49s
7:	learn: 90.9145536	total: 2.96s	remaining: 1h 1m 38s
8:	learn: 89.6846397	total: 3.32s	remaining: 1h 1m 26s
9:	learn: 88.4743632	total: 3.67s	remaining: 1h 1m 5s
10:	learn: 87.2838888	total: 4.1s	remaining: 1h 2m 2s
11:	learn: 86.1068307	total: 4.44s	remaining: 1h 1m 39s
12:	learn: 84.9475415	total: 4.79s	remaining: 1h 1m 21s
13:	learn: 83.8137384	total: 5.15s	remaining: 1h 1m 13s
14:	learn: 82.6894440	total: 5.5s	remaining: 1h 57s
15:	learn: 81.5848249	total: 5.86s	remaining: 1h 58s
16:	learn: 80.4981882	total: 6.19s	remaining: 1h 36s
17:	learn: 79.4289236	total: 6.54s	remaining: 1h 27s
18:	learn: 78.368950

"\nparams_catboost = {\n    'iterations': [100, 200, 300, 500, 1000, 3000, 10000],\n    'learning_rate': [0.001, 0.005, 0.01, 0.015, 0.02, 0.03, 0.05, 0.1],\n    'depth': range(1, 10, 1),\n    'subsample': [x / 100.0 for x in range(5, 100, 10)],\n    'colsample_bylevel': [x / 100.0 for x in range(5, 100, 10)],\n    'min_data_in_leaf': range(1, 100, 10),\n    'l2_leaf_reg': [1, 3, 5, 7, 9],\n    #'min_child_samples': [1, 4, 8, 16, 32]\n}\n\ngrid_search_catboost = RandomizedSearchCV(\n    catboost_model,\n    cv = TimeSeriesSplit(n_splits=4),\n    param_distributions=params_catboost,\n    n_jobs=-1,\n    random_state=random_state,\n    scoring='neg_mean_absolute_error'\n)\n\ngrid_search_catboost.fit(features_train, target_train)"

In [52]:
cat_predict_test = catboost_model.predict(feat_xgb_test)
cat_predict_train = catboost_model.predict(feat_xgb_train)

In [53]:
mae_train, mape_train, r2_train = metrics_hour(target_all_train, cat_predict_train )
mae_open_test, mape_open_test, r2_open_test = metrics_hour(target_open_test, cat_predict_test )

results = pd.concat([results,
pd.DataFrame([[f'тренировочная CAT {FEATURES}', mae_train, mape_train, r2_train], [f'тестовая CAT {FEATURES}', mae_open_test, mape_open_test, r2_open_test]], 
             columns=('Выборка', 'MAE', 'MAPE', 'R2'))
 ])

/opt/anaconda3/envs/p311/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/anaconda3/envs/p311/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/opt/anaconda3/envs/p311/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/anaconda3/envs/p311/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_

In [54]:
importance = xgb_model.feature_importances_
# предположим, что 'X' - это ваши данные
feature_name = feat_xgb_test.columns
# создание DataFrame
importance_df_xgb = pd.DataFrame({'feature': feature_name, 'importance': importance})
# сортировка по важности
importance_df_xgb = importance_df_xgb.sort_values(by='importance', ascending=False)


In [55]:
importance_df_xgb

,feature,importance
8,target_lag_24,0.469577
196,day_of_week_6,0.016503
132,day_of_week_4,0.011926
228,day_of_week_7,0.011528
676,day_of_week_21,0.010968
...,...,...
273,has_rain_probability_8,0.000000
482,year_15,0.000000
465,has_rain_probability_14,0.000000
290,year_9,0.000000


In [56]:
merged_df = pd.merge(importance_df_lgbm, importance_df_xgb, on='feature', how='outer', suffixes=('_lgbm', '_xgb'))

In [57]:
merged_df

,feature,importance_lgbm,importance_xgb
0,target_lag_24,5168,0.469577
1,target_lag_336,3262,0.006438
2,target_lag_72,2998,0.002726
3,target_lag_24_1,2148,0.000919
4,temp_pred,2112,0.000611
...,...,...,...
793,year_14,2,0.000217
794,year_10,1,0.000538
795,preholidays_8,1,NaN
796,preholidays_14,1,NaN


In [58]:
# Замена NaN на 0 в столбце 'importance_xgb'
merged_df['importance_xgb'].fillna(0, inplace=True)

# Нормализация важности признаков
merged_df['importance_lgbm'] = merged_df['importance_lgbm'] / merged_df['importance_lgbm'].max()
merged_df['importance_xgb'] = merged_df['importance_xgb'] / merged_df['importance_xgb'].max()

# Создание столбца 'importance_ensemble', который является средним значением 'importance_lgbm' и 'importance_xgb'
merged_df['importance_ensemble'] = (merged_df['importance_lgbm'] + merged_df['importance_xgb']) / 2

In [59]:
merged_df

,feature,importance_lgbm,importance_xgb,importance_ensemble
0,target_lag_24,1.000000,1.000000,1.000000
1,target_lag_336,0.631192,0.013711,0.322452
2,target_lag_72,0.580108,0.005806,0.292957
3,target_lag_24_1,0.415635,0.001956,0.208795
4,temp_pred,0.408669,0.001300,0.204985
...,...,...,...,...
793,year_14,0.000387,0.000462,0.000425
794,year_10,0.000193,0.001145,0.000669
795,preholidays_8,0.000193,0.000000,0.000097
796,preholidays_14,0.000193,0.000000,0.000097


In [60]:
!pip install openpyxl

In [61]:
merged_df.to_excel('feature_importance.xlsx')

### 4 Объединяем результаты ансамбля моделей

In [62]:
predict_simple_ensemble_train = (xgb_predict_train + l_predict_train)/2
predict_simple_ensemble_test = (xgb_predict_test + l_predict_test)/2

In [63]:
mae_train, mape_train, r2_train = metrics_hour(target_all_train, predict_simple_ensemble_train)
mae_open_test, mape_open_test, r2_open_test = metrics_hour(target_open_test, predict_simple_ensemble_test)

results_ensemble = results
results_ensemble = pd.concat([results_ensemble,
pd.DataFrame([[f'тренировочная ансамбля LGBM и XGB  {FEATURES}', mae_train, mape_train, r2_train], [f'тестовая ансамбля LGBM и XGB {FEATURES}', mae_open_test, mape_open_test, r2_open_test]], 
             columns=('Выборка', 'MAE', 'MAPE', 'R2'))
 ])

/opt/anaconda3/envs/p311/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/anaconda3/envs/p311/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/opt/anaconda3/envs/p311/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/anaconda3/envs/p311/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_

In [64]:
display(results_ensemble)

,Выборка,MAE,MAPE,R2
0,тренировочная LGBM _Aug_27_alpha_3,3.403002,0.007062,0.996599
1,тестовая LGBM _Aug_27_alpha_3,6.092151,0.014274,0.985890
0,тренировочная XGB _Aug_27_alpha_3,3.828241,0.008054,0.997552
1,тестовая XGB _Aug_27_alpha_3,6.141930,0.014345,0.984693
0,тренировочная CAT _Aug_27_alpha_3,2.389001,0.005083,0.999066
1,тестовая CAT _Aug_27_alpha_3,6.003542,0.014254,0.986401
0,тренировочная ансамбля LGBM и XGB _Aug_27_alp...,3.502858,0.007314,0.997425
1,тестовая ансамбля LGBM и XGB _Aug_27_alpha_3,5.964305,0.013936,0.986072


In [65]:
predict_simple_ensemble_train = (cat_predict_train + l_predict_train)/2
predict_simple_ensemble_test = (cat_predict_test + l_predict_test)/2

In [66]:
mae_train, mape_train, r2_train = metrics_hour(target_all_train, predict_simple_ensemble_train)
mae_open_test, mape_open_test, r2_open_test = metrics_hour(target_open_test, predict_simple_ensemble_test)


results_ensemble = pd.concat([results_ensemble,
pd.DataFrame([[f'тренировочная ансамбля CatBoost и LGBM  {FEATURES}', mae_train, mape_train, r2_train], [f'тестовая ансамбля CatBoost и LGBM {FEATURES} ', mae_open_test, mape_open_test, r2_open_test]], 
             columns=('Выборка', 'MAE', 'MAPE', 'R2'))
 ])

/opt/anaconda3/envs/p311/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/anaconda3/envs/p311/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/opt/anaconda3/envs/p311/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/anaconda3/envs/p311/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_

In [67]:
display(results_ensemble)

,Выборка,MAE,MAPE,R2
0,тренировочная LGBM _Aug_27_alpha_3,3.403002,0.007062,0.996599
1,тестовая LGBM _Aug_27_alpha_3,6.092151,0.014274,0.985890
0,тренировочная XGB _Aug_27_alpha_3,3.828241,0.008054,0.997552
1,тестовая XGB _Aug_27_alpha_3,6.141930,0.014345,0.984693
0,тренировочная CAT _Aug_27_alpha_3,2.389001,0.005083,0.999066
1,тестовая CAT _Aug_27_alpha_3,6.003542,0.014254,0.986401
0,тренировочная ансамбля LGBM и XGB _Aug_27_alp...,3.502858,0.007314,0.997425
1,тестовая ансамбля LGBM и XGB _Aug_27_alpha_3,5.964305,0.013936,0.986072
0,тренировочная ансамбля CatBoost и LGBM _Aug_2...,2.789836,0.005842,0.998304
1,тестовая ансамбля CatBoost и LGBM _Aug_27_alph...,5.851637,0.013786,0.987096


In [78]:
predict_simple_ensemble_train = xgb_predict_train*0.3 + l_predict_train*0.3 + cat_predict_train*0.4
predict_simple_ensemble_test = xgb_predict_test*0.3 + l_predict_test*0.3 + cat_predict_test*0.4

In [79]:
mae_train, mape_train, r2_train = metrics_hour(target_all_train, predict_simple_ensemble_train)
mae_open_test, mape_open_test, r2_open_test = metrics_hour(target_open_test, predict_simple_ensemble_test)


results_ensemble = pd.concat([results_ensemble,
pd.DataFrame([[f'тренировочная ансамбля CatBoost, LGBM и XGBoost {FEATURES}', mae_train, mape_train, r2_train], [f'тестовая ансамбля CatBoost, LGBM и XGBoost {FEATURES}', mae_open_test, mape_open_test, r2_open_test]], 
             columns=('Выборка', 'MAE', 'MAPE', 'R2'))
 ])

/opt/anaconda3/envs/p311/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/anaconda3/envs/p311/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/opt/anaconda3/envs/p311/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/anaconda3/envs/p311/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_

In [80]:
display(results_ensemble)

,Выборка,MAE,MAPE,R2
0,тренировочная LGBM _Aug_27_alpha_3,3.403002,0.007062,0.996599
1,тестовая LGBM _Aug_27_alpha_3,6.092151,0.014274,0.985890
0,тренировочная XGB _Aug_27_alpha_3,3.828241,0.008054,0.997552
1,тестовая XGB _Aug_27_alpha_3,6.141930,0.014345,0.984693
0,тренировочная CAT _Aug_27_alpha_3,2.389001,0.005083,0.999066
1,тестовая CAT _Aug_27_alpha_3,6.003542,0.014254,0.986401
0,тренировочная ансамбля LGBM и XGB _Aug_27_alp...,3.502858,0.007314,0.997425
1,тестовая ансамбля LGBM и XGB _Aug_27_alpha_3,5.964305,0.013936,0.986072
0,тренировочная ансамбля CatBoost и LGBM _Aug_2...,2.789836,0.005842,0.998304
1,тестовая ансамбля CatBoost и LGBM _Aug_27_alph...,5.851637,0.013786,0.987096


In [71]:
import os
#num = 3
# определите путь к папке, которую вы хотите создать
folder_path = "new_models"

# проверьте, существует ли уже папка
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

lgbm_model_all_train.booster_.save_model(f'{folder_path}/lgb_model{FEATURES}_{num}.txt')
xgb_model.save_model(f'{folder_path}/xgb_model{FEATURES}_{num}.json')
catboost_model.save_model(f'{folder_path}/catboost_model{FEATURES}_{num}.cbm', format="cbm")

In [72]:
#FEATURES = 'BASE'
results.to_csv(f'{folder_path}/results_LGBM_XGBoost_Catboost_ensemble_{FEATURES}_{num}')

In [74]:
pd.DataFrame(xgb_predict_train).to_csv(f'{folder_path}/xgb_predict_train_{FEATURES}_{num}.csv', index=False)
pd.DataFrame(xgb_predict_test).to_csv(f'{folder_path}/xgb_predict_test_{FEATURES}_{num}.csv', index=False)

In [75]:
pd.DataFrame(l_predict_train).to_csv(f'{folder_path}/lgbm_predict_train_{FEATURES}_{num}.csv', index=False)
pd.DataFrame(l_predict_test).to_csv(f'{folder_path}/lgbm_predict_test_{FEATURES}_{num}.csv', index=False)

In [76]:
pd.DataFrame(cat_predict_train).to_csv(f'{folder_path}/cat_predict_train_{FEATURES}_{num}.csv', index=False)
pd.DataFrame(cat_predict_test).to_csv(f'{folder_path}/cat_predict_test_{FEATURES}_{num}.csv', index=False)

In [77]:
target_all_train.to_csv(f'{folder_path}/target_train_{FEATURES}_{num}.csv', index=False)
target_open_test.to_csv(f'{folder_path}/target_test{FEATURES}_{num}.csv', index=False)